I started this competition investigating neural networks with this kernel https://www.kaggle.com/mulargui/keras-nn
Now switching to using ensembles in this new kernel. As of today V6 is the most performant version.
You can find all my notes and versions at https://github.com/mulargui/kaggle-Classify-forest-types

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#load data
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

####### DATA PREPARATION #####
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

# one data set with all features
X = pd.concat([x,x_predict],keys=[0,1])

In [2]:
###### FEATURE ENGINEERING #####
#https://www.kaggle.com/evimarp/top-6-roosevelt-national-forest-competition
from itertools import combinations
from bisect import bisect
X['Euclidean_distance_to_hydro'] = (X.Vertical_Distance_To_Hydrology**2 
                                         + X.Horizontal_Distance_To_Hydrology**2)**.5

cols = [
        'Horizontal_Distance_To_Roadways',
        'Horizontal_Distance_To_Fire_Points',
        'Horizontal_Distance_To_Hydrology',
]
X['distance_mean'] = X[cols].mean(axis=1)
X['distance_sum'] = X[cols].sum(axis=1)
X['distance_road_fire'] = X[cols[:2]].mean(axis=1)
X['distance_hydro_fire'] = X[cols[1:]].mean(axis=1)
X['distance_road_hydro'] = X[[cols[0], cols[2]]].mean(axis=1)
    
X['distance_sum_road_fire'] = X[cols[:2]].sum(axis=1)
X['distance_sum_hydro_fire'] = X[cols[1:]].sum(axis=1)
X['distance_sum_road_hydro'] = X[[cols[0], cols[2]]].sum(axis=1)
    
X['distance_dif_road_fire'] = X[cols[0]] - X[cols[1]]
X['distance_dif_hydro_road'] = X[cols[2]] - X[cols[0]]
X['distance_dif_hydro_fire'] = X[cols[2]] - X[cols[1]]
    
# Vertical distances measures
colv = ['Elevation', 'Vertical_Distance_To_Hydrology']
X['Vertical_dif'] = X[colv[0]] - X[colv[1]]
X['Vertical_sum'] = X[colv].sum(axis=1)
    
SHADES = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    
X['shade_noon_diff'] = X['Hillshade_9am'] - X['Hillshade_Noon']
X['shade_3pm_diff'] = X['Hillshade_Noon'] - X['Hillshade_3pm']
X['shade_all_diff'] = X['Hillshade_9am'] - X['Hillshade_3pm']
X['shade_sum'] = X[SHADES].sum(axis=1)
X['shade_mean'] = X[SHADES].mean(axis=1)
  
X['ElevationHydro'] = X['Elevation'] - 0.25 * X['Euclidean_distance_to_hydro']
X['ElevationV'] = X['Elevation'] - X['Vertical_Distance_To_Hydrology']
X['ElevationH'] = X['Elevation'] - 0.19 * X['Horizontal_Distance_To_Hydrology']

X['Elevation2'] = X['Elevation']**2
X['ElevationLog'] = np.log1p(X['Elevation'])

X['Aspect_cos'] = np.cos(np.radians(X.Aspect))
X['Aspect_sin'] = np.sin(np.radians(X.Aspect))
#df['Slope_sin'] = np.sin(np.radians(df.Slope))
X['Aspectcos_Slope'] = X.Slope * X.Aspect_cos
#df['Aspectsin_Slope'] = df.Slope * df.Aspect_sin
    
cardinals = [i for i in range(45, 361, 90)]
points = ['N', 'E', 'S', 'W']
X['Cardinal'] = X.Aspect.apply(lambda x: points[bisect(cardinals, x) % 4])

d = {'N': 0, 'E': 1, 'S': 0, 'W':-1}
X['Cardinal'] = X.Cardinal.apply(lambda x: d[x])

#adding features based on https://douglas-fraser.com/forest_cover_management.pdf pages 21,22
#note: not all climatic and geologic codes have a soil type
columns=['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6']
X['Climatic2'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type7', 'Soil_Type8']
X['Climatic3'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13']
X['Climatic4'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type14', 'Soil_Type15']
X['Climatic5'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type16', 'Soil_Type17', 'Soil_Type18']
X['Climatic6'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
    'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
    'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34']
X['Climatic7'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
X['Climatic8'] = np.select([X[columns].sum(1).gt(0)], [1])

columns=['Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type19', 'Soil_Type20',
    'Soil_Type21']
X['Geologic1'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type9', 'Soil_Type22', 'Soil_Type23']
X['Geologic2'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type7', 'Soil_Type8']
X['Geologic5'] = np.select([X[columns].sum(1).gt(0)], [1])
columns=['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
    'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type18', 'Soil_Type24',
    'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
    'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 
    'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']
X['Geologic7'] = np.select([X[columns].sum(1).gt(0)], [1])

#Reversing One-Hot-Encoding to Categorical attributes, several articles recommend it for decision tree algorithms
#Doing it for Soil_Type, Wilderness_Area, Geologic and Climatic
#we are also replacing the categorical values by random numbers to difficult to the algorythm to find relationships between the values
X['Tmp']=np.where(X.loc[:, 'Soil_Type1':'Soil_Type40'])[1] +1
cols = [c for c in X.columns if c[:9] != 'Soil_Type']
X=X[cols]
X=X.rename(columns = {'Tmp':'Soil_Type'})

min=X['Soil_Type'].min()
max=X['Soil_Type'].max()
s=np.random.randint(low=1, high=np.iinfo(np.int64).max, size=(max-min+1), dtype='int64')
for i in range (min,max+1):
    X['Soil_Type'].replace(to_replace=i, value=s[i-min], inplace=True)

X['Tmp']=np.where(X.loc[:, 'Wilderness_Area1':'Wilderness_Area4'])[1] +1
cols = [c for c in X.columns if c[:15] != 'Wilderness_Area']
X=X[cols]
X=X.rename(columns = {'Tmp':'Wilderness_Area'})

min=X['Wilderness_Area'].min()
max=X['Wilderness_Area'].max()
s=np.random.randint(low=1, high=np.iinfo(np.int64).max, size=(max-min+1), dtype='int64')
for i in range (min,max+1):
    X['Wilderness_Area'].replace(to_replace=i, value=s[i-min], inplace=True)

X['Tmp']=np.where(X.loc[:, 'Climatic2':'Climatic8'])[1] +1
cols = [c for c in X.columns if c[:8] != 'Climatic']
X=X[cols]
X=X.rename(columns = {'Tmp':'Climatic'})

min=X['Climatic'].min()
max=X['Climatic'].max()
s=np.random.randint(low=1, high=np.iinfo(np.int64).max, size=(max-min+1), dtype='int64')
for i in range (min,max+1):
    X['Climatic'].replace(to_replace=i, value=s[i-min], inplace=True)

X['Tmp']=np.where(X.loc[:, 'Geologic1':'Geologic7'])[1] +1
cols = [c for c in X.columns if c[:8] != 'Geologic']
X=X[cols]
X=X.rename(columns = {'Tmp':'Geologic'})

min=X['Geologic'].min()
max=X['Geologic'].max()
s=np.random.randint(low=1, high=np.iinfo(np.int64).max, size=(max-min+1), dtype='int64')
for i in range (min,max+1):
    X['Geologic'].replace(to_replace=i, value=s[i-min], inplace=True)

#https://www.kaggle.com/arateris/2-layer-k-fold-learning-forest-cover 
# Add PCA features
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99).fit(X)
trans = pca.transform(X)

for i in range(trans.shape[1]):
    col_name= 'pca'+str(i+1)
    X[col_name] = trans[:,i]

# Adding Gaussian Mixture features to perform some unsupervised learning hints from the full data
#https://www.kaggle.com/arateris/2-layer-k-fold-learning-forest-cover 
#https://www.kaggle.com/stevegreenau/stacking-multiple-classifiers-clustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
gmix = GaussianMixture(n_components=10) 
gaussian = gmix.fit_predict(StandardScaler().fit_transform(X))
X['GM'] = gaussian

# Scale and bin features
from sklearn.preprocessing import MinMaxScaler
X.loc[:, :] = np.floor(MinMaxScaler((0, 100)).fit_transform(X))
X = X.astype('int8')

In [3]:
#break it down again in train and test
x,x_predict = X.xs(0),X.xs(1)

In [5]:
###### THIS IS THE ENSEMBLE SECTION ######
#https://www.kaggle.com/kwabenantim/forest-cover-stacking-multiple-classifiers
import random
from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

randomstate = 1
random.seed(randomstate)
np.random.seed(randomstate)

ab_clf = AdaBoostClassifier(n_estimators=200,
                            base_estimator=DecisionTreeClassifier(
                                min_samples_leaf=2,
                                random_state=randomstate),
                            random_state=randomstate)

#max_features = min(30, x.columns.size)
max_features = 30
et_clf = ExtraTreesClassifier(n_estimators=300,
                              min_samples_leaf=2,
                              min_samples_split=2,
                              max_depth=50,
                              max_features=max_features,
                              random_state=randomstate,
                              n_jobs=1)

lg_clf = LGBMClassifier(n_estimators=300,
                        num_leaves=128,
                        verbose=-1,
                        random_state=randomstate,
                        n_jobs=1)

rf_clf = RandomForestClassifier(n_estimators=300,
                                random_state=randomstate,
                                n_jobs=1)

#Added a KNN classifier to the ensemble
#https://www.kaggle.com/edumunozsala/feature-eng-and-a-simple-stacked-model
knn_clf = KNeighborsClassifier(n_neighbors=y.nunique(), n_jobs=1)

ensemble = [('AdaBoostClassifier', ab_clf),
            ('ExtraTreesClassifier', et_clf),
            ('LGBMClassifier', lg_clf),
            ('KNNClassifier', knn_clf),
            ('RandomForestClassifier', rf_clf)
]

#Cross-validating classifiers
for label, clf in ensemble:
    score = cross_val_score(clf, x, y,
                            cv=5,
                            scoring='accuracy',
                            verbose=0,
                            n_jobs=-1)
# Fitting stack
stack = StackingCVClassifier(classifiers=[ab_clf, et_clf, lg_clf, knn_clf, rf_clf],
                             meta_classifier=rf_clf,
                             cv=5,
                             stratify=True,
                             shuffle=True,
                             use_probas=True,
                             use_features_in_secondary=True,
                             verbose=1,
                             random_state=randomstate)

stack = stack.fit(x, y)

y_predict = stack.predict(x_predict)
y_predict = pd.Series(y_predict, index=x_predict.index, dtype=y.dtype)

Fitting 5 classifiers...
Fitting classifier1: adaboostclassifier (1/5)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: extratreesclassifier (2/5)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier3: lgbmclassifier (3/5)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier4: kneighborsclassifier (4/5)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier5: randomforestclassifier (5/5)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   52.5s finished


In [6]:
# Save predictions to a file for submission
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict})
output.to_csv('submission.csv', index=False)

#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv